## Simulation of one round of blockchain-based federated learning 

In [1]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [2]:
import torch
print(f"Cuda available: {torch.cuda.is_available()}")
# Get the name of the CUDA device
print(torch.cuda.get_device_name(0))

try:
    print(
        f"major and minor cuda capability of the device: {torch.cuda.get_device_capability()}")
except Exception:
    print("No Cuda available")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Check if CUDA is available and set the default tensor type to CUDA
print('Using device: %s' % device)
if torch.cuda.is_available():
    torch.set_default_device('cuda')
    print("Cuda set as default device")
else:
    torch.set_default_device('cpu')
    print("Cuda not available, CPU set as default device")

/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400430266/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Cuda available: False


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

### Config variables for models and training


In [3]:
from os.path import join as pjoin
import json
from client.train import train
from client.model.perceptron import MultiLayerPerceptron
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json,  weights_init, update_lr
from client.aggregators import federated_aggregate
from client.dataloader import get_cifar10_dataloaders, get_cifar10_datasets

input_size = 32 * 32 * 3
hidden_size = [50]
num_classes = 10
num_epochs = 1
learning_rate = 1e-3
learning_rate_decay = 0.95
reg = 0.001
modelpath = 'client/models/'
train_flag = True


ImportError: cannot import name 'get_file_path' from 'client.utils' (/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/utils.py)

### Load data


In [4]:

root = 'client/data/'
num_training = 49000
num_validation = 1000
batch_size = 200
train_dataset, val_dataset, test_dataset = get_cifar10_datasets(
    root, num_training, num_validation)
train_loader, val_loader, test_loader = get_cifar10_dataloaders(
    root, batch_size, num_training, num_validation, device)

dataloaders = {
    'train': train_loader,
    'validation': val_loader,
    'test': test_loader
}


Files already downloaded and verified
Files already downloaded and verified


### Create two local models


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from client.model.perceptron import Perceptron

model1 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model1.to(device)
print(model1)
model1_name = 'bcfl_model1'

model2 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model2.to(device)
print(model2)
model2_name = 'bcfl_model2'




MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)
MultiLayerPerceptron(
  (layers): Sequential(
    (0): Linear(in_features=3072, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=10, bias=True)
  )
)


### Train first local model


In [6]:

model1_name = 'bcfl_model1'
# Training
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model1.parameters(), lr=learning_rate, weight_decay=reg)
# optimizer.to(device)

train(model1, modelpath, model1_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)



Epoch [1/1], Step [1/245], Loss: 2.3026
Epoch [1/1], Step [2/245], Loss: 2.3011
Epoch [1/1], Step [3/245], Loss: 2.2946
Epoch [1/1], Step [4/245], Loss: 2.2808
Epoch [1/1], Step [5/245], Loss: 2.2647
Epoch [1/1], Step [6/245], Loss: 2.2555
Epoch [1/1], Step [7/245], Loss: 2.2303
Epoch [1/1], Step [8/245], Loss: 2.2195
Epoch [1/1], Step [9/245], Loss: 2.2171
Epoch [1/1], Step [10/245], Loss: 2.1714
Epoch [1/1], Step [11/245], Loss: 2.1580
Epoch [1/1], Step [12/245], Loss: 2.1134
Epoch [1/1], Step [13/245], Loss: 2.0887
Epoch [1/1], Step [14/245], Loss: 2.1647
Epoch [1/1], Step [15/245], Loss: 2.0902
Epoch [1/1], Step [16/245], Loss: 2.1081
Epoch [1/1], Step [17/245], Loss: 1.9998
Epoch [1/1], Step [18/245], Loss: 1.9906
Epoch [1/1], Step [19/245], Loss: 2.0200
Epoch [1/1], Step [20/245], Loss: 2.0716
Epoch [1/1], Step [21/245], Loss: 1.9121
Epoch [1/1], Step [22/245], Loss: 1.9942
Epoch [1/1], Step [23/245], Loss: 2.0938
Epoch [1/1], Step [24/245], Loss: 1.8950
Epoch [1/1], Step [25/245

### Train second local model

In [63]:
model2_name = 'bcfl_model4'
# Training
model2.apply(weights_init)
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model2.parameters(), lr=learning_rate, weight_decay=reg)

train(model2, modelpath, model2_name, dataloaders, criterion, optimizer,
      learning_rate, learning_rate_decay, input_size, num_epochs, device)

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Test the trained models

In [7]:
from os.path import join as pjoin
modelpath = 'client/models/'

# Run the test code once you have your by setting train flag to false
# and loading the best model
model_ckpt = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model_ckpt = torch.load(pjoin(modelpath, model1_name + '.ckpt'))
model1.load_state_dict(model_ckpt)
model = model1
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))
    
    

model = model2
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        ####################################################

        # reshape images to input size
        images = images.reshape(-1, input_size).to(device)
        # set the model for evaluation
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total == 1000:
            break

    print('Accuracy of the network on the {} test images: {} %'.format(
        total, 100 * correct / total))

Accuracy of the network on the 1000 test images: 42.5 %
Accuracy of the network on the 1000 test images: 10.5 %



### Forward the two local models to the blockchain


In [8]:
# Send the saved model to Fabric-SDK via Gateway Client (REST call)

from client.services.gateway_client import submit_model, get_all_models, get_model


print(submit_model(model1_name, model1.state_dict()))

# print(submit_model(model2_name, model2.state_dict()))



{'message': 'Model submitted succesfully'}


### Submit empty model to the blockchain

In [9]:
# TODO this should be triggered automatically when the total users in the blockchain have triggered their updates

model3 = MultiLayerPerceptron(input_size, hidden_size, num_classes)
model3.to(device)
model3.zero_init()
model3_name = 'bcfl_model_empty'
# print(model3)
submit_model(model3_name, model3.state_dict())


{'message': 'Model submitted succesfully'}


### Aggregate the local models on the blockchain
TODO this should be triggered automatically when the total users in the blockchain have triggered their updates


In [10]:
# TODO this should be triggered automatically when the total users in the blockchain have triggered their updates

from client.services.gateway_client import aggregate_models



aggregate_models([model1_name, model3_name])


{'message': 'Models aggregated succesfully'}


### Download the new global model


In [11]:

from os.path import join as pjoin
import json
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json, deserialize_model_msgpack

# retrieve future global model and convert it again to pytorch model

# TODO change
model_avg_name = model1_name + 'and' + model3_name
model_data = get_model(model_avg_name)
decoded_state_dict = deserialize_model_msgpack(model_data['modelParams'])

global_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
global_model.to(device)
global_model.load_state_dict(decoded_state_dict)



<All keys matched successfully>

### Print the new global model and compare it

In [14]:
from client.utils import count_parameters, compare_models


# print(global_model)
   
print(model1.state_dict()['layers.2.bias'])
print(global_model.state_dict()['layers.2.bias'])
# the averaged model should be half the model1


print(count_parameters(model1))
print(count_parameters(global_model))


tensor([-0.0142, -0.0449,  0.0383,  0.0089,  0.0415,  0.0235, -0.0104, -0.0061,
         0.0021, -0.0400], device='cuda:0')
tensor([-0.0071, -0.0225,  0.0192,  0.0044,  0.0207,  0.0117, -0.0052, -0.0030,
         0.0011, -0.0200], device='cuda:0')
154160
154160


### Test the new global model

In [21]:

from client.test import test_cifar


model = global_model
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
test_cifar(model, input_size, test_loader, device)


Accuracy of the network on the 1000 test images: 42.3 %


### Retrieve global model and convert it again to pytorch model

In [ ]:

from os.path import join as pjoin
import json
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json, count_parameters, compare_models


local_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
local_model = torch.load(pjoin(modelpath,  'ml_model2.ckpt'))
model.load_state_dict(local_model)

model_params = get_model('ml_model2')
# print(client_response)


bc_model = MultiLayerPerceptron(input_size, hidden_size, num_classes)
bc_model.to(device)
# print(bc_model)

load_model_from_json(bc_model, model_params)
print(bc_model)
print(count_parameters(model))
print(count_parameters(bc_model))
print(compare_models(model, bc_model))
# for parameter in model.parameters():
#     print(parameter)


## Transformers

### Create Bert Tiny model

In [4]:
from os.path import join as pjoin

from transformers import AutoModel # For BERTs
from transformers import AutoModelForSequenceClassification # For models fine-tuned on MNLI
from transformers import AutoTokenizer
import torch

# tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny") # v1 and v2
bert_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny") # v1 and v2
print(bert_model)
bert_modelname = 'bert_model'

# tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny") # v1 and v2
bert_model2 = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny") # v1 and v2
bert_modelname2 = 'bert_model2'


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

### Save the model in a local file, load it again and compare

In [4]:
# Send the saved model to Fabric-SDK via Gateway Client (REST call)

from client.services.gateway_client import submit_model, get_all_models, get_model

modelpath = 'client/models/'

# Save the model checkpoint
torch.save(bert_model.state_dict(), pjoin(modelpath, bert_modelname + '.pt'))
    
loaded_bert_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny") # v1 and v2

loaded_bert_model_params = torch.load(pjoin(modelpath, bert_modelname + '.pt'))
loaded_bert_model.load_state_dict(loaded_bert_model_params)

print(count_parameters(bert_model))
print(count_parameters(loaded_bert_model))
print(compare_models(bert_model, loaded_bert_model))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'count_parameters' is not defined

### Serialize and encode model, decode and try if its equal

In [6]:

import sys
from client.services.gateway_client import submit_model
from client.services.utils import serialize_model_numpy, deserialize_model_numpy, print_layer_size, serialize_model_msgpack, deserialize_model_msgpack
from client.utils import compare_models, compare_state_dicts, compare_weights
from client.model.bert_tiny import get_bert_tiny_model
from client.model.bert_mini import get_bert_mini_model
from client.model.bert_small import get_bert_small_model
from client.model.bert_medium import get_bert_medium_model
from client.model.distilbert_base import get_distilbert_base_model

# encoded = serialize_model_msgpack(bert_model.state_dict(), 0, 0)
# # encoded = serialize_model_numpy(bert_model.state_dict())

# print(sys.getsizeof(encoded) / 1024 / 1024)
# print(type(encoded))
# decoded_state_dict = deserialize_model_msgpack(encoded)
# # decoded_state_dict = deserialize_model_numpy(encoded)



# bert2_model = get_bert_tiny_model('cpu')
# bert2_model.load_state_dict(decoded_state_dict)


# print(compare_state_dicts(bert_model.state_dict(), bert2_model.state_dict()))
# print(compare_models(bert_model, bert2_model))
# print(compare_weights(bert_model, bert2_model))

# bert_tiny_model = get_bert_tiny_model('cpu')
# encoded_tiny = serialize_model_msgpack(bert_tiny_model.state_dict(), 0, 0)
# print(sys.getsizeof(encoded_tiny) / 1024 / 1024)

# bert_mini_model = get_bert_mini_model('cpu')
# encoded_mini = serialize_model_msgpack(bert_mini_model.state_dict(), 0, 0)
# print(sys.getsizeof(encoded_mini) / 1024 / 1024)

# bert_small_model = get_bert_small_model('cpu')
# encoded_small = serialize_model_msgpack(bert_small_model.state_dict(), 0, 0)
# print(sys.getsizeof(encoded_small) / 1024 / 1024)


# bert_medium_model = get_bert_medium_model('cpu')
# encoded_medium = serialize_model_msgpack(bert_medium_model.state_dict(), 0, 0)
# print(sys.getsizeof(encoded_medium) / 1024 / 1024)


distilbert_base_model = get_distilbert_base_model('cpu')
encoded_distilbert_base = serialize_model_msgpack(distilbert_base_model.state_dict(), 0, 0)
print(sys.getsizeof(encoded_distilbert_base) / 1024 / 1024)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Submit Bert Tiny model to the blockchain

In [16]:




import sys

from client.services.gateway_client import submit_model
from client.utils import print_layer_size, serialize_model_msgpack, deserialize_model_msgpack, compare_models, compare_state_dicts, compare_weights

# Send the saved model to Fabric-SDK via Gateway Client (REST call)
# print(submit_model(bert_modelname, bert_model.state_dict()))
print(submit_model(bert_modelname, bert_model.state_dict(), 0, 0))
print(submit_model(bert_modelname2, bert_model2.state_dict(), 0, 0))

# print_layer_size(bert_model.state_dict(), 0, 6, 'mb', 'base64')
# print(submit_compressed_model(bert_modelname, bert_model.state_dict()))


{'message': 'Model submitted succesfully'}
{'message': 'Model submitted succesfully'}


### Download the model and load it again


In [37]:
from os.path import join as pjoin
import json
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json, count_parameters, compare_models




blockchain_bert_model = get_model(bert_modelname)

decoded_state_dict = deserialize_model_msgpack(blockchain_bert_model['modelParams'])


bert2_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny") # v1 and v2

bert2_model.load_state_dict(decoded_state_dict)

print(compare_state_dicts(bert_model.state_dict(), bert2_model.state_dict()))
print(compare_models(bert_model, bert2_model))
print(compare_weights(bert_model, bert2_model))




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True
True
True
4386178


NameError: name 'bc_bert_model' is not defined

### Aggregate the bert models on the blockchain

In [19]:
# TODO this should be triggered automatically when the total users in the blockchain have triggered their updates

from client.services.gateway_client import aggregate_models



aggregate_models([bert_modelname, bert_modelname2])


{'message': 'Models aggregated succesfully'}

### Download the new global BERT model


In [8]:

from os.path import join as pjoin
import json
from client.services.gateway_client import submit_model, get_all_models, get_model
from client.utils import load_model_from_json, deserialize_model_msgpack

# retrieve future global model and convert it again to pytorch model

bert_global_model_name = bert_modelname + 'and' + bert_modelname2
bert_global_data = get_model(bert_global_model_name)
decoded_state_dict = deserialize_model_msgpack(bert_global_data['modelParams'])


bert_global_model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny") # v1 and v2
bert_global_model.load_state_dict(decoded_state_dict)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

### Print the new global model and compare it

In [17]:
from client.utils import count_parameters, compare_models


# print(global_model)
   
print(bert_model.state_dict()['classifier.weight'])
print(bert_global_model.state_dict()['classifier.weight'])
# the averaged model should be half the model1


print(count_parameters(bert_model))
print(count_parameters(bert_global_model))


tensor([[-0.0176, -0.0025,  0.0031,  0.0403,  0.0071, -0.0137,  0.0155,  0.0092,
         -0.0148,  0.0160, -0.0336,  0.0085, -0.0220, -0.0046,  0.0186, -0.0057,
          0.0012,  0.0070, -0.0008, -0.0072, -0.0223,  0.0316,  0.0265,  0.0123,
         -0.0250,  0.0260,  0.0037, -0.0412,  0.0500,  0.0091, -0.0261, -0.0048,
         -0.0104,  0.0037,  0.0301, -0.0183,  0.0080, -0.0004, -0.0162,  0.0089,
         -0.0075,  0.0042, -0.0013,  0.0015,  0.0108, -0.0135, -0.0197,  0.0087,
         -0.0028,  0.0166,  0.0035, -0.0078,  0.0092, -0.0178,  0.0275, -0.0143,
         -0.0245, -0.0093,  0.0155, -0.0292,  0.0071,  0.0064, -0.0435,  0.0062,
         -0.0222,  0.0182,  0.0045, -0.0125,  0.0027,  0.0191,  0.0496, -0.0256,
         -0.0383, -0.0020, -0.0140,  0.0282,  0.0127,  0.0414,  0.0132, -0.0198,
          0.0092,  0.0149, -0.0034,  0.0033, -0.0270, -0.0232, -0.0431, -0.0148,
          0.0134,  0.0216,  0.0276,  0.0171,  0.0036,  0.0227,  0.0279,  0.0063,
          0.0101, -0.0112,  

In [59]:
import torch
# NVIDIA drivers not working
torch.set_default_device('cpu')
device = ('cpu')